# 2. Retrieval

The main objective here is to query our dataset of nutrional information from a given list of food names and their quantity.

1. Find a way to store the data into a suitable databse:
2. Explore the dataset
3. Find a way to match food names with the ones found in the database
4. Find a way to output the nutrional value for the given amount

In [ ]:
import pandas as pd
import pyarrow.parquet as pq 

pf = pq.ParquetFile('data/food.parquet')

df = next(pf.iter_batches(1000)).to_pandas()
df

The dataset contains multiple languages. For the sake of simplicity, let's filter out any rows with language other than english